In [ ]:

"""
Projeto: Detecção Automática de Pragas em Plantações de Café com CNN (N1)
Disciplina: Inteligência Artificial – 7ºN CC - Noite
Professor: Prof. Dr. Ivan Carlos Alcântara de Oliveira

Integrantes:
- [Preencha: Nome Completo] – [RA] – [email]
- [Adicione mais linhas se necessário]

Síntese do notebook:
- Estrutura padrão do relatório N1 (título, resumo, introdução, fundamentação, problema, ética).
- Download/organização do dataset (IP102 como base, opcional JMuBEN/JMuBEN2).
- Análise exploratória (contagem por classe, exemplos visuais), preparação dos dados (normalização, augmentation).
- Metodologia e resultados esperados (sem treino pesado aqui; fica para N2).

Histórico de alterações:
- 2025-09-28 – [Seu Nome] – Criação do notebook de N1 com EDA e preparação.
"""


In [ ]:
Professor: Prof. Dr. Ivan Carlos Alcântara de Oliveira

Integrantes:
- [Preencha: Nome Completo] – [RA] – [email]
- [Adicione mais linhas se necessário]

Síntese do notebook:
- Estrutura padrão do relatório N1 (título, resumo, introdução, fundamentação, problema, ética).
- Download/organização do dataset (IP102 como base, opcional JMuBEN/JMuBEN2).
- Análise exploratória (contagem por classe, exemplos visuais), preparação dos dados (normalização, augmentation).
- Metodologia e resultados esperados (sem treino pesado aqui; fica para N2).

Histórico de alterações:
- 2025-09-28 – [Seu Nome] – Criação do notebook de N1 com EDA e preparação.
"""


# Detecção Automática de Pragas em Plantações de Café com CNN (Parte 2 – N1)

Universidade Presbiteriana Mackenzie  
Faculdade de Computação e Informática  
*Disciplina:* Inteligência Artificial – 7ºN CC - Noite  
*Professor:* Prof. Dr. Ivan Carlos Alcântara de Oliveira  

## 1) Título
*Detecção Automática de Pragas em Plantações de Café Utilizando Redes Neurais Convolucionais*

## 2) Integrantes (nome, e-mail, RA)
Nome do Aluno	               / RA             
Eduardo Honorio Friaça	       10408959

Felipe Jiao	                   10408852

Hao Yue Zheng	               10408948

Samuel Zheng	               10395781


## 3) Resumo
Este trabalho descreve a proposta e a análise exploratória de um sistema de Visão Computacional para classificação de pragas em imagens de plantações de café. Utilizamos como base o IP102 (75k imagens, 102 classes de pragas) e, opcionalmente, conjuntos específicos de café (JMuBEN/JMuBEN2). São apresentados: origem dos dados, organização e limpeza, preparo (redimensionamento, normalização, data augmentation), e uma análise de distribuição de classes. Também são detalhados os aspectos éticos e a metodologia com transfer learning (ResNet18/MobileNet), além dos resultados esperados (acurácia ≥ 70%).

## 4) Introdução
### a) Contextualização
Pragas agrícolas geram perdas econômicas relevantes. No café, insetos como broca-do-cafeeiro e bicho-mineiro são recorrentes. A IA pode acelerar o diagnóstico.
### b) Justificativa
Diagnóstico manual é demorado e sujeito a erro. Classificação automática por imagens auxilia a tomada de decisão no campo.
### c) Objetivo
Preparar e explorar dados para um classificador de pragas em café, estabelecendo uma base reprodutível para o treino do modelo (N2).
### d) Opção do projeto
Opção ML/DL/VC/PLN – Visão Computacional / Deep Learning.

## 5) Fundamentação Teórica (Resumida)
CNNs extraem padrões visuais via convoluções; transfer learning permite ajustar modelos pré-treinados (ResNet18/MobileNetV2) a novas classes com poucos dados.

## 6) Descrição do Problema
Identificar automaticamente a praga presente na imagem de folha/fruto de café, apesar de variações de iluminação, foco e estágio da infestação.

## 7) Aspectos Éticos e Responsabilidade
- Uso dos dados conforme licenças; anonimização quando necessário.
- Transparência de limitações (falsos positivos/negativos); o sistema não substitui especialistas.
- Incentivo a manejo sustentável; evitar decisões automatizadas sem supervisão.


## 8) Dataset, Análise Exploratória e Preparação (Python)
*Fontes principais:*  
- *IP102*: ~75k imagens, 102 classes (cobre a exigência de ≥10k imagens).  
- *(Opcional)* JMuBEN/JMuBEN2: 58.5k imagens de folhas de café (5 classes).  

### 8.1 Instalação de dependências

In [ ]:


import os, zipfile, tarfile, shutil, random
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm

print("Versões:")
print("  torch:", torch.__version__)


### 8.2 Download e organização do IP102
O bloco abaixo tenta baixar espelhos do IP102. *Caso falhe*, baixe manualmente do repositório oficial e extraia para `data_ip102/`.  
- Repo oficial: https://github.com/xpwu95/IP102
- Após extrair, aponte `IMAGES_DIR` para a pasta `images/` com subpastas por classe.

In [ ]:

DATA_ROOT = Path("./data_ip102")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

def try_download_gdown(file_name, file_id):
    try:
        import gdown
    except ImportError:
        print("Instale gdown se quiser baixar automático: !pip install gdown")
        return False
    out = DATA_ROOT / file_name
    if out.exists():
        print(file_name, "já existe.")
        return True
    url = f"https://drive.google.com/uc?id={file_id}"
    try:
        print("Baixando:", file_name)
        gdown.download(url, str(out), quiet=False)
        return out.exists()
    except Exception as e:
        print("Falha no download:", e)
        return False

def extract_any(p):
    p = Path(p）
    if p.exists():
        if p.suffix == ".zip":
            with zipfile.ZipFile(p, "r") as zf:
                zf.extractall(DATA_ROOT)
        elif p.suffix in [".tar", ".gz", ".tgz", ".bz2"]:
            with tarfile.open(p, "r:*") as tf:
                tf.extractall(DATA_ROOT)

attempts = [
    ("IP102_Images.zip", "1gGqf8c4c2h1R5xQX_SAMPLE_REPLACE_WITH_VALID_ID"),
]
for fn, fid in attempts:
    ok = try_download_gdown(fn, fid)
    if ok:
        extract_any(DATA_ROOT / fn)

IMAGES_DIR = DATA_ROOT / "images"  
if not IMAGES_DIR.exists():
    print("ATENÇÃO: Extraia manualmente o IP102 para", IMAGES_DIR, "com estrutura: images/class_name/*.jpg")
else:
    print("Pasta de imagens encontrada:", IMAGES_DIR)


### 8.3 Preparação (transformações) e carregamento da estrutura ImageFolder

In [ ]:

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_WORKERS = 2

data_tfms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
    'eval': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]),
}

if IMAGES_DIR.exists():
    full_ds = datasets.ImageFolder(IMAGES_DIR, transform=data_tfms['eval'])
    class_to_idx = full_ds.class_to_idx
    idx_to_class = {v:k for k,v in class_to_idx.items()}
    print("Total de imagens:", len(full_ds))
    print("Total de classes:", len(class_to_idx))
else:
    full_ds = None
    print("Configure IMAGES_DIR primeiro.")


### 8.4 Análise Exploratória: contagem por classe

In [ ]:

if full_ds is not None:
    counts = {c:0 for c in class_to_idx}
    for _, y in full_ds.samples:
        counts[idx_to_class[y]] += 1
    df_counts = pd.DataFrame.from_dict(counts, orient='index', columns=['count']).sort_values('count', ascending=False)
    display(df_counts.head(20))
    print("Classes com menos imagens (potencial desbalanceamento):")
    display(df_counts.tail(20))

    # Gráfico: distribuição das top-N classes (N=20), um gráfico único sem subplots
    topN = 20
    df_top = df_counts.head(topN)
    plt.figure(figsize=(10,6))
    plt.bar(df_top.index, df_top['count'])
    plt.title(f"Distribuição (Top {topN}) de Imagens por Classe")
    plt.xticks(rotation=90)
    plt.xlabel("Classe")
    plt.ylabel("Quantidade de imagens")
    plt.tight_layout()
    plt.show()


### 8.5 Visualização de exemplos (sem subplots)
Mostraremos algumas imagens individuais (uma por figura) para inspecionar a qualidade dos dados.

In [ ]:

def show_n_examples_per_class(ds, max_classes=5):
    if ds is None:
        print("Dataset não carregado.")
        return
    shown_classes = 0
    seen = set()
    for path, y in ds.samples:
        cname = idx_to_class[y]
        if cname in seen:
            continue
        seen.add(cname)
        try:
            img = Image.open(path).convert('RGB')
            plt.figure(figsize=(4,4))
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"Classe: {cname}")
            plt.show()
            shown_classes += 1
            if shown_classes >= max_classes:
                break
        except Exception as e:
            continue

if full_ds is not None:
    show_n_examples_per_class(full_ds, max_classes=5)


### 8.6 Split (treino/validação/teste)
Divisão estratificada simples por classe (70/15/15).

In [ ]:

import random, math

if full_ds is not None:
    samples_by_class = {}
    for p, y in full_ds.samples:
        cname = idx_to_class[y]
        samples_by_class.setdefault(cname, []).append((p,y))

    train_list, val_list, test_list = [], [], []
    for cname, items in samples_by_class.items():
        random.shuffle(items)
        n = len(items)
        n_train = int(0.7*n)
        n_val = int(0.15*n)
        train_list += items[:n_train]
        val_list   += items[n_train:n_train+n_val]
        test_list  += items[n_train+n_val:]

    class SubsetDS(Dataset):
        def __init__(self, samples, transform, loader):
            self.samples = samples
            self.transform = transform
            self.loader = loader
        def __len__(self):
            return len(self.samples)
        def __getitem__(self, i):
            path, y = self.samples[i]
            img = Image.open(path).convert('RGB')
            if self.transform: 
                img = self.transform(img)
            return img, y

    ds_train = SubsetDS(train_list, data_tfms['train'], full_ds.loader)
    ds_val   = SubsetDS(val_list,   data_tfms['eval'],  full_ds.loader)
    ds_test  = SubsetDS(test_list,  data_tfms['eval'],  full_ds.loader)

    dl_train = DataLoader(ds_train, batch_size=32, shuffle=True, num_workers=2)
    dl_val   = DataLoader(ds_val,   batch_size=32, shuffle=False, num_workers=2)
    dl_test  = DataLoader(ds_test,  batch_size=32, shuffle=False, num_workers=2)

    print(f"Tamanhos -> train: {len(ds_train)}, val: {len(ds_val)}, test: {len(ds_test)}")
else:
    print("Carregue o dataset antes do split.")


## 9) Metodologia e Resultados Esperados
- *Modelo:* Transfer learning (ResNet18/MobileNetV2) com última camada ajustada para o número de classes.
- *Treino:* Acontecerá no N2 (outro notebook focado em desenvolvimento e resultados). Nesta etapa, reportamos apenas o *plano*.
- *Métricas esperadas:* Acurácia ≥ 70% no conjunto de teste; além disso, precisão/recall/F1 por classe e matriz de confusão.
- *Próximos passos (N2):* Treinar, validar, ajustar hiperparâmetros, gerar relatório final, vídeo e link GitHub públicos.

## 10) Referências (citar dentro do texto do relatório)
- IP102: A Large-Scale Benchmark Dataset for Insect Pest Recognition (repositório oficial).
- He, K. et al. Deep Residual Learning for Image Recognition (ResNet).
- Howard, A. et al. MobileNets: Efficient Convolutional Neural Networks for Mobile Vision.

## 11) Bibliografia
- Goodfellow, Bengio, Courville – Deep Learning (MIT Press)
- Géron – Hands-On Machine Learning (O’Reilly)
- Bishop – Pattern Recognition and Machine Learning (Springer)
